In [3]:
from DataLoader import importar_ventas,importar_cluster_stores,leer_csv,agregar_fourier,preparar_test
from DataLoader import crear_archivo_kaggle
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import r2_score

In [4]:
ventas=importar_ventas()

In [5]:
ventas

,date,demand,mean_price,std_price,max_price,min_price,mean_discount,mean_diff_factory,std_discount,max_discount,...,min_diff_factory,size,group,category,cluster,store_cod,subgroup_cod,day,month,year
4056384,2021-01-10,1.000000,319.510000,0.000000,319.51,319.51,-0.017057,-0.240148,0.000000,-0.017057,...,-0.240148,1.0,Computing,Electronics,0,0,0,10,1,2021
1635397,2021-01-13,8.000041,241.460000,0.000000,241.46,241.46,-0.060742,-0.529973,0.038716,-0.022778,...,-0.993703,3.0,Computing,Electronics,0,0,0,13,1,2021
3447047,2021-01-18,4.000000,250.180000,0.000000,250.18,250.18,0.012871,-0.169803,0.000000,0.012871,...,-0.169803,4.0,Computing,Electronics,0,0,0,18,1,2021
3853530,2021-01-22,5.000000,268.270000,0.000000,268.27,268.27,0.029215,-0.187797,0.010120,0.046744,...,-0.300475,4.0,Computing,Electronics,0,0,0,22,1,2021
241952,2021-01-23,3.000000,297.770000,0.000000,297.77,297.77,0.010243,-0.113674,0.000000,0.010243,...,-0.113674,3.0,Computing,Electronics,0,0,0,23,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741754,2023-12-15,2.000000,27.320000,0.000000,27.32,27.32,0.067716,-0.106606,0.000000,0.067716,...,-0.106606,2.0,Games,Toys,8,156,73,15,12,2023
4175882,2023-12-16,15.000593,29.456667,3.035845,33.75,27.31,-0.044976,-0.403494,0.038643,-0.008056,...,-0.570251,6.0,Games,Toys,8,156,73,16,12,2023
5099248,2023-12-21,3.999622,26.450000,0.000000,26.45,26.45,0.049397,-0.180057,0.017459,0.074088,...,-0.270060,3.0,Games,Toys,8,156,73,21,12,2023
4444785,2023-12-26,2.000000,26.530000,0.000000,26.53,26.53,-0.006408,-0.233237,0.000000,-0.006408,...,-0.233237,2.0,Games,Toys,8,156,73,26,12,2023


In [6]:
test=preparar_test("ids_test.csv")
#test

In [7]:
agregar_fourier(ventas,2)
agregar_fourier(test,2)

In [8]:

FEATURES=[
    #'mean_price', 
    #'cluster', 
    #'store_cod', 
    #'subgroup_cod',
    #'demand_lag_1', #'demand_lag_2', 'demand_lag_3', 
    #'demand_lag_4','demand_lag_5', 'demand_lag_6', 'demand_lag_7', 'day', 'month', 
    'year',
   'anual_sin_1', 
    'anual_cos_1',
   # 'mensual_sin_1', 
    #'mensual_cos_1',
   #'anual_sin_2', 
    #'anual_cos_2',
    'mensual_sin_2', 
    #'mensual_cos_2',
    "std_price",
    "max_price", 
   "min_price"
] #recordar que ya probamos con fourier 3 y 4 y fue un fracaso

TARGET=['demand']

test_interno=ventas.groupby(["subgroup_cod","store_cod"]).tail(7)
train=ventas
train_interno=ventas.drop(test_interno.index)

X_train=train[FEATURES]
y_train=train[TARGET]
X_test_real=test[FEATURES]

X_train_interno=train_interno[FEATURES]
y_train_interno=train_interno[TARGET]

X_test_interno=test_interno[FEATURES]
y_test_interno=test_interno[TARGET]

# X_test_2023=test_2023[FEATURES]
# y_test_2023=test_2023[TARGET]

# X_train_2023=train_2023[FEATURES]
# y_train_2023=train_2023[TARGET]

In [9]:
"""
# Parameters
initial_train_weeks = 130   # ~2.4 years of training data
horizon_weeks = 1           # predict 1 week ahead
early_stopping_rounds = 50  # patience for early stopping

# Store results
all_scores = []

# Pre-calculate week-year identifiers (optimized)
ventas['week_year'] = (ventas['date'].dt.isocalendar().year * 100 + 
                       ventas['date'].dt.isocalendar().week)
all_weeks = sorted(ventas['week_year'].unique())

# Ensure data is sorted by date to prevent leakage
ventas = ventas.sort_values('date')

# Rolling window validation
for i in range(initial_train_weeks, len(all_weeks) - horizon_weeks + 1):
    print(i)
    train_weeks = all_weeks[:i]
    val_weeks = all_weeks[i:i + horizon_weeks]
    
    # Create masks using the pre-calculated week_year
    train_mask = ventas['week_year'].isin(train_weeks)
    val_mask = ventas['week_year'].isin(val_weeks)
    
    # Data splitting
    X_train = ventas.loc[train_mask, FEATURES]
    y_train = ventas.loc[train_mask, TARGET].values.ravel()
    X_val = ventas.loc[val_mask, FEATURES]
    y_val = ventas.loc[val_mask, TARGET].values.ravel()
    
    # Train model with your custom r2_score_lgb
    reg_lgb = lgb.LGBMRegressor(
        n_estimators=600,
        learning_rate=0.3,
        max_depth=8,
        objective='regression',
        verbose=-1  # Silent mode
    )
    
    reg_lgb.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric=r2_score_lgb,  # Using your pre-defined metric
        callbacks=[
            lgb.early_stopping(stopping_rounds=early_stopping_rounds, verbose=False),
            lgb.log_evaluation(0)  # Disable logging if verbose=-1
        ]
    )
    
    # Predict and store R2 score
    y_pred = reg_lgb.predict(X_val)
    all_scores.append(r2_score(y_val, y_pred))

# Final results
print(f"Average R² over {len(all_scores)} folds: {np.mean(all_scores):.4f}")
print(f"Std R²: {np.std(all_scores):.4f}")

all_scores
"""

'\n# Parameters\ninitial_train_weeks = 130   # ~2.4 years of training data\nhorizon_weeks = 1           # predict 1 week ahead\nearly_stopping_rounds = 50  # patience for early stopping\n\n# Store results\nall_scores = []\n\n# Pre-calculate week-year identifiers (optimized)\nventas[\'week_year\'] = (ventas[\'date\'].dt.isocalendar().year * 100 + \n                       ventas[\'date\'].dt.isocalendar().week)\nall_weeks = sorted(ventas[\'week_year\'].unique())\n\n# Ensure data is sorted by date to prevent leakage\nventas = ventas.sort_values(\'date\')\n\n# Rolling window validation\nfor i in range(initial_train_weeks, len(all_weeks) - horizon_weeks + 1):\n    print(i)\n    train_weeks = all_weeks[:i]\n    val_weeks = all_weeks[i:i + horizon_weeks]\n    \n    # Create masks using the pre-calculated week_year\n    train_mask = ventas[\'week_year\'].isin(train_weeks)\n    val_mask = ventas[\'week_year\'].isin(val_weeks)\n    \n    # Data splitting\n    X_train = ventas.loc[train_mask, FEA

In [11]:
reg_lgb = lgb.LGBMRegressor(
    n_estimators=600, #deberia parar con el early stopping
    learning_rate=0.3, 
    max_depth=8,
    objective='regression',
    # colsample_bytree=0.9,
    verbose=1  
)

def r2_score_lgb(y_true, y_pred):
    return 'r2', r2_score(y_true, y_pred), True 

reg_lgb.fit(
    X_train, y_train,
    eval_set=[(X_train_interno, y_train_interno)], 
    eval_metric=r2_score_lgb,
    callbacks=[lgb.early_stopping(stopping_rounds=50),lgb.log_evaluation(50) ]
)
                    

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1283
[LightGBM] [Info] Number of data points in the train set: 5206793, number of used features: 7
[LightGBM] [Info] Start training from score 5.233525
Training until validation scores don't improve for 50 rounds
[50]	valid_0's l2: 8.21809	valid_0's r2: 0.440102
[100]	valid_0's l2: 8.08769	valid_0's r2: 0.448986
[150]	valid_0's l2: 8.0249	valid_0's r2: 0.453264
[200]	valid_0's l2: 7.98334	valid_0's r2: 0.456096
[250]	valid_0's l2: 7.95164	valid_0's r2: 0.458255
[300]	valid_0's l2: 7.92431	valid_0's r2: 0.460117
[350]	valid_0's l2: 7.89861	valid_0's r2: 0.461868
[400]	valid_0's l2: 7.87724	valid_0's r2: 0.463324
[450]	valid_0's l2: 7.85693	valid_0's r2: 0.464708
[500]	valid_0's l2: 7.84093	valid_0's r2: 0.465798
[550]	valid_0's l2: 7.82

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,8
,learning_rate,0.3
,n_estimators,600
,subsample_for_bin,200000
,objective,'regression'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [12]:
feature_importance = pd.DataFrame(
    reg_lgb.feature_importances_,
    columns=["importance"],
    index=reg_lgb.feature_name_
).sort_values(by="importance", ascending=False)

feature_importance

,importance
std_price,4058
max_price,3584
min_price,3542
anual_sin_1,2851
anual_cos_1,2287
year,1261
mensual_sin_2,417


In [14]:
prediccion=reg_lgb.predict(X_test_real)

kaggle=crear_archivo_kaggle(prediccion,test)

In [47]:
kaggle.columns
(kaggle["STORE_SUBGROUP_DATE_ID"]==kaggle_correcto["store_subgroup_date_id"]).sum()

np.int64(14)

In [48]:
test=leer_csv("ids_test")
test=test.merge(kaggle_correcto, on="store_subgroup_date_id", how="left")
(kaggle["STORE_SUBGROUP_DATE_ID"]==test["store_subgroup_date_id"]).sum() # ahi anda bien

np.int64(77700)

In [50]:
kaggle.describe()

,PREDICCION
count,77700.000000
mean,4.412285
std,1.835059
min,3.090258
25%,3.321192
50%,3.464810
75%,5.614933
max,16.243299


In [51]:
y_test_interno.describe()

,demand
count,77672.000000
mean,4.316356
std,2.983366
min,1.000000
25%,2.000000
50%,4.000000
75%,5.000000
max,45.000000


KeyError: 'sku'